In [2]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd

d:\Projects\Text Classification\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
bert_custom_1 = '../models/bert_custom_1'

In [4]:
tokenizer = BertTokenizer.from_pretrained(bert_custom_1)
model = BertForSequenceClassification.from_pretrained(bert_custom_1)

In [5]:
df = pd.read_json('../data/clean_data.json', orient="records")

In [6]:
text = df['text'][0]
label = df['label_encoded'][0]

In [7]:
text

'6. Conditions to Obligations. The several obligations of the Underwriters are subject to the following conditions: (a) No stop order suspending the effectiveness of the Registration Statement shall be in effect, and no proceedings for such purpose shall be pending before or threatened by the Commission. (b) The representations and warranties of the Company contained in this Agreement shall be true and correct as of the date hereof and as of the Closing Date, and the Company shall have complied with all of the agreements and satisfied all of the conditions on its part to be performed or satisfied hereunder on or before the Closing Date. (c) Subsequent to the execution and delivery of this Agreement and prior to the Closing Date: (i) there shall not have occurred any downgrading, nor shall any notice have been given of any intended or potential downgrading or of any review for a possible change (other than a notice of a possible upgrading), in the rating accorded any of the securities o

In [8]:
label

2

In [9]:
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
inputs = {k: v.to(device) for k, v in inputs.items()}

In [13]:
with torch.no_grad():
  outputs = model(**inputs)

d:\Projects\Text Classification\venv\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [14]:
predicted_labels = torch.argmax(outputs.logits, dim=-1)

In [15]:
predicted_labels

tensor([2], device='cuda:0')

In [16]:
label

2

In [28]:
df_unlabeled = pd.read_json('../data/unlabeled_data.json', orient="records")

In [71]:
text = df_unlabeled["text"].iloc[4]

In [82]:
text = '| Three months ended March 31,| Three months ended March 31,| Three months ended March 31,| Three months ended March 31,|. | 2023| 2023| 2022| 2022|. NET INCOME| | $132.1| | $160.9|. OTHER COMPREHENSIVE INCOME| | | | |. Foreign currency translation:| | | | |. Foreign currency translation adjustments| 22.3| 22.3| (3.6)| (3.6)|. Derivative instruments:| | | | |. Changes in fair value of derivative instruments| (2.8)| (2.8)| 5.9| 5.9|. Recognition of previously unrealized net gain in net income| (0.4)| (0.4)| (0.3)| (0.3)|. Income tax effect| 0.8| 0.8| (1.4)| (1.4)|. | (2.4)| (2.4)| 4.2| 4.2|. Defined benefit pension and other postretirement plans:| | | | |. Net actuarial gain (loss) for the period| 1.9| 1.9| (0.6)| (0.6)|. Amortization of unrecognized losses, transition obligation and prior service cost included in net income| 1.7| 1.7| 1.7| 1.7|. Other| 0.4| 0.4| (0.1)| (0.1)|. Income tax effect| (0.4)| (0.4)| 0.6| 0.6|. | 3.6| 3.6| 1.6| 1.6|. Other comprehensive income, net of tax| 23.5| 23.5| 2.2| 2.2|. TOTAL COMPREHENSIVE INCOME| 155.6| 155.6| 163.1| 163.1|. Less: comprehensive income attributable to non-controlling interests| 5.8| 5.8| 1.8| 1.8|. COMPREHENSIVE INCOME ATTRIBUTABLE TO IPG| | $149.8| | $161.3|'

In [113]:
text = '1.Capitalized Terms. All capitalized terms used and not defined in this Amendment shall have the meanings given thereto in the Plan.'

In [114]:
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

In [115]:
inputs = {k: v.to(device) for k, v in inputs.items()}

In [116]:
with torch.no_grad():
  outputs = model(**inputs)

In [117]:
predicted_labels = torch.argmax(outputs.logits, dim=-1)
predicted_labels

tensor([2], device='cuda:0')

In [118]:
res = predicted_labels.cpu().tolist()
res

[2]

In [119]:
import pickle

In [120]:
with open('../models/encoder_model.pkl', 'rb') as f:
  label_encoder = pickle.load(f)

In [121]:
decoded_labels = label_encoder.inverse_transform(res)
decoded_labels

array(['TEXT'], dtype=object)

In [122]:
text

'1.Capitalized Terms. All capitalized terms used and not defined in this Amendment shall have the meanings given thereto in the Plan.'